In [168]:
from collections import defaultdict
class Graph:
    def __init__(self, connections, directed):
        self.graph = defaultdict(set)
        self.directed = directed
        self.add_connections(connections)
        
    def add_connections(self, connections):
        for node1, node2 in connections:
            self.add(node1, node2)
                
    def add(self, node1, node2):
        self.graph[node1].add(node2)
        if not self.directed:
            self.graph[node2].add(node1)
    def remove(self, node):
        for source, targets in self.graph.items():
            try:
                targets.remove(node)
            except KeyError:
                pass
        try:
            del self.graph[node]
        except KeyError:
            pass
        
    def removeEdge(self, edge):
        source, target = edge
        try:
            self.graph[source].remove(target)
        except KeyError:
            pass
        if not self.directed:
            try:
                self.graph[target].remove(source)
            except KeyError:
                pass
        
    def nodes(self):
        nodes = set()
        for source, targets in self.graph.items():
            nodes.add(source)
            nodes.update(targets)
        return list(nodes)
    
    def edges(self, sourceSpecified=None):
        edges = []
        if sourceSpecified:
            for target in self.graph[sourceSpecified]:
                edges.append((sourceSpecified, target))
        else:
            for source in self.graph:
                for target in self.graph[source]:
                    edges.append((source, target))
                    if not self.directed:
                        try:
                            edges.remove((target, source))
                        except ValueError:
                            pass
        return edges

In [170]:
import unittest
class TestGraph(unittest.TestCase):
    def setUp(self):
        self.connections = [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('C','F'), ('F', 'G')]
        self.graph = Graph(connections, directed=True)
    
    def test_add_connections(self):
        newConnections = [('G', 'H'), ('G', 'A')]
        self.graph.add_connections(newConnections)
        gConnections = self.graph.graph['G']
        self.assertEqual(len(gConnections), 2)
        self.assertTrue('H' in gConnections and 'A' in gConnections)
        
    def test_add(self):
        self.graph.add('G','H')
        gConnections = self.graph.graph['G']
        self.assertEqual(len(gConnections), 1)
        self.assertTrue('H' in gConnections)
        
    def test_nodes(self):
        expectedNodes = set(['A','B','C','D','E','F','G'])
        nodes = self.graph.nodes()
        self.assertEqual(set(nodes), expectedNodes)
    def test_remove(self):
        self.graph.remove('B')
        aConnections = self.graph.graph['A']
        self.assertEqual(len(aConnections), 0)
        self.assertTrue('B' not in self.graph.graph)
        
    def test_removeEdge(self):
        self.graph.removeEdge(('A','B'))
        edgesFromA = self.graph.edges('A')
        self.assertEqual(edgesFromA, [])
    
    def test_edges(self):
        edges = self.graph.edges()
        for edge in edges:
            self.assertTrue(edge in self.connections)
        edgesFromA = self.graph.edges('A')
        self.assertEqual(edgesFromA, [('A','B')])
        
testSuiteGraph = unittest.TestLoader().loadTestsFromTestCase(TestGraph)
runner = unittest.TextTestRunner()
runner.run(testSuiteGraph)

......
----------------------------------------------------------------------
Ran 6 tests in 0.010s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

In [95]:
def breadthFirstSearch(graph, start):
    nodesAtLevel = [start]
    visited = []
    while len(nodesAtLevel) > 0:
        nodesAtNextLevel = []
        yield nodesAtLevel
        for node in nodesAtLevel:
            targets = graph.graph[node]
            for target in targets:
                if target not in visited:
                    nodesAtNextLevel.append(target)
            visited.append(node)
        nodesAtLevel = nodesAtNextLevel
        
        
def depthFirstSearch(graph, start):
    nodesToVisit = [start]
    visited = []
    while len(nodesToVisit) > 0:
        node = nodesToVisit.pop()
        yield node
        targets = graph.graph[node]
        for target in targets:
            if target not in visited:
                nodesToVisit.append(target)
        visited.append(node)

In [96]:
import unittest
class TestSearch(unittest.TestCase):
    def setUp(self):
        connections = [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('C','F'), ('F', 'G')]
        self.graph = Graph(connections, directed=True)
    
    def test_breadthFirstSearch(self):
        index = 0
        nodesAtLevels = [['A'],['B'],['C'],['D','F'],['E','G']]
        for nodes in breadthFirstSearch(self.graph, 'A'):
            self.assertEqual(nodes, nodesAtLevels[index])
            index += 1
            
    def test_depthFirstSearch(self):
        visited = set()
        for node in depthFirstSearch(self.graph, 'A'):
            visited.add(node)
        self.assertEqual(len(visited), 7)
        
testSuiteSearch = unittest.TestLoader().loadTestsFromTestCase(TestSearch)
runner = unittest.TextTestRunner()
runner.run(testSuiteSearch)

..
----------------------------------------------------------------------
Ran 2 tests in 0.009s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [97]:
# 4.1 Route Between Nodes: Given a directed graph, design an algorithm to find out whether there is a route between two nodes.

# breadth first or depth first search starting from one node and looking for the other
# time complexity: O(V+E)
# space complexity: O(V)
        
def routeBetweenNodes(graph, start, end):
    for nodes in breadthFirstSearch(graph, start):
        if end in nodes:
            return True
    return False

In [98]:
import unittest
class TestRouteBetweenNodes(unittest.TestCase):
    def setUp(self):
        connections = [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('C','F'), ('F', 'G')]
        self.graph = Graph(connections, directed=True)
    
    def test_routeBetweenNodes(self):
        self.assertTrue(routeBetweenNodes(self.graph, 'A', 'G'))
        self.assertFalse(routeBetweenNodes(self.graph, 'D', 'B'))
        
testSuiteRouteBetweenNodes = unittest.TestLoader().loadTestsFromTestCase(TestRouteBetweenNodes)
runner = unittest.TextTestRunner()
runner.run(testSuiteRouteBetweenNodes)

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [133]:
class BinaryTreeNode:
    def __init__(self, data):
        self.data = data
        self.parent = None
        self.leftChild = None
        self.rightChild = None
    def getLeftChild(self):
        return self.leftChild
    def getRightChild(self):
        return self.rightChild
    def setRightChild(self, node):
        self.rightChild = node
    def setLeftChild(self, node):
        self.leftChild = node
    def getData(self):
        return self.data
    def getParent(self):
        return self.parent
    def setParent(self, parent):
        self.parent = parent

In [135]:
import unittest
class TestBinaryTreeNode(unittest.TestCase):
    def setUp(self):
        self.node1 = BinaryTreeNode(1)
        self.node2 = BinaryTreeNode(2)
        self.node3 = BinaryTreeNode(3)
        self.node1.setLeftChild(self.node2)
        self.node2.setParent(self.node1)
    
    def test_getLeftChild(self):
        self.assertEqual(self.node1.getLeftChild(), self.node2)
        self.assertTrue(self.node2.getLeftChild() is None)
        
    def test_getRightChild(self):
        self.assertTrue(self.node1.getRightChild() is None)
        
    def test_setRightChild(self):
        self.node1.setRightChild(self.node3)
        self.assertEqual(self.node1.getRightChild().getData(), 3)
        
    def test_setLeftChild(self):
        self.node1.setLeftChild(self.node3)
        self.assertEqual(self.node1.getLeftChild().getData(), 3)
        
    def test_getData(self):
        self.assertEqual(self.node1.getData(), 1)
        
    def test_getParent(self):
        self.assertEqual(self.node2.getParent(), self.node1)
    
    def test_setParent(self):
        self.node3.setParent(self.node2)
        self.assertEqual(self.node3.getParent(), self.node2)
        
testSuiteBinaryTreeNode = unittest.TestLoader().loadTestsFromTestCase(TestBinaryTreeNode)
runner = unittest.TextTestRunner()
runner.run(testSuiteBinaryTreeNode)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.017s

OK


<unittest.runner.TextTestResult run=7 errors=0 failures=0>

In [136]:
class BinaryTree:
    def __init__(self, data=[]):
        self.root = None
        for datum in data:
            self.add(datum)
    
    def add(self, data):
        newNode = BinaryTreeNode(data)
        
        if self.root:
            curNode = self.root
            while curNode:
                if data > curNode.getData():
                    if not curNode.getRightChild():
                        curNode.setRightChild(newNode)
                        newNode.setParent(curNode)
                        break
                    else:
                        curNode = curNode.getRightChild()
                else:
                    if not curNode.getLeftChild():
                        curNode.setLeftChild(newNode)
                        newNode.setParent(curNode)
                        break
                    else:
                        curNode = curNode.getLeftChild()
        else:
            self.root = newNode

    def search(self, data):
        return self.search_recursive(self.root, data)
        
    def search_recursive(self, node, data):
        if node:
            if node.getData() == data:
                return node
            if data > node.getData():
                return self.search_recursive(node.getRightChild(), data)
            else:
                return self.search_recursive(node.getLeftChild(), data)
        else:
            return None
    def inOrderTraversal(self, node, function):
        if node:
            self.inOrderTraversal(node.getLeftChild(), function)
            function(node.getData())
            self.inOrderTraversal(node.getRightChild(), function)
            
    def print(self):
        self.inOrderTraversal(self.root, print)
        
    def height(self):
        return self.height_recursive(self.root)
    def height_recursive(self, node):
        if not node:
            return 0
        leftHeight = self.height_recursive(node.getLeftChild())
        rightHeight = self.height_recursive(node.getRightChild())
        return max(leftHeight, rightHeight) + 1

In [137]:
import unittest
class TestBinaryTree(unittest.TestCase):
    def setUp(self):
        self.binaryTree = BinaryTree([5,4,6,3,7,1,8])
    def test_add(self):
        found = self.binaryTree.search(2)
        self.assertTrue(found is None)
        self.binaryTree.add(2)
        found = self.binaryTree.search(2)
        self.assertEqual(found.getData(), 2)
        
    def test_search(self):
        found = self.binaryTree.search(1)
        self.assertTrue(found is not None)
        self.assertEqual(found.getData(), 1)
        found = self.binaryTree.search(9)
        self.assertTrue(found is None)
        
    def test_height(self):
        self.assertEqual(self.binaryTree.height(), 4)
        self.binaryTree.add(0)
        self.assertEqual(self.binaryTree.height(), 5)
        
testSuiteBinaryTree = unittest.TestLoader().loadTestsFromTestCase(TestBinaryTree)
runner = unittest.TextTestRunner()
runner.run(testSuiteBinaryTree)

...
----------------------------------------------------------------------
Ran 3 tests in 0.009s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [103]:
# 4.2 Minimal Tree: Given a sorted (increasing order) array with unique integer elements, write an algorithm to create a binary search tree with minimal height.

# find middle element and add it to tree
# repeat with left and right half

def minimalTree(array):
    binarySearchTree = BinaryTree()
    minimalTree_recursive(binarySearchTree, array)
    return binarySearchTree
def minimalTree_recursive(tree, array):
    if len(array) == 0:
        return
    middleIndex = len(array) // 2
    tree.add(array[middleIndex])
    lowerLow = 0
    lowerHigh = middleIndex
    if lowerLow < lowerHigh:
        minimalTree_recursive(tree, array[lowerLow:lowerHigh])
    higherLow = middleIndex + 1
    higherHigh = len(array)
    if higherLow < higherHigh:
        minimalTree_recursive(tree, array[higherLow:higherHigh])

In [104]:
import unittest
from math import log
class TestMinimalTree(unittest.TestCase):
    def setUp(self):
        # array must be sorted in ascending order
        self.array = [3,4,6,7,8,14]
        
    def test_minimalTree(self):
        tree = minimalTree(self.array)
        expectedMinHeight = round(log(len(self.array), 2))
        self.assertEqual(tree.height(), expectedMinHeight)
        
testSuiteMinimalTree = unittest.TestLoader().loadTestsFromTestCase(TestMinimalTree)
runner = unittest.TextTestRunner()
runner.run(testSuiteMinimalTree)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [105]:
class LinkedListNode:
    def __init__(self, value):
        self.value = value
        self.nextNode = None
    def getNextNode(self):
        return self.nextNode
    def setNextNode(self, node):
        self.nextNode = node
    def getValue(self):
        return self.value
    def setValue(self, value):
        self.value = value
        
class LinkedList:
    head = None
    def __init__(self, values):
        try:
            for value in reversed(values):
                self.add(value)
        except TypeError:
            if type(values) is LinkedListNode:
                self.head = values
            else:
                self.head = LinkedListNode(values)
    def add(self, data):
        newNode = LinkedListNode(data)
        newNode.setNextNode(self.head)
        self.head = newNode
    def get(self, index):
        node = self.head
        data = None
        for i in range(index):
            if node:
                node = node.getNextNode()
            else:
                break
        if node:
            data = node.getValue()
        return data
    def print(self):
        print(self.getList())
    def length(self):
        node = self.head
        length = 0
        while node:
            length += 1
            node = node.getNextNode()
        return length
    def getList(self):
        node = self.head
        nodes = []
        while node is not None:
            nodes.append(node.getValue())
            node = node.getNextNode()
        return nodes

In [106]:
# 4.3 List of Depths: Given a binary tree, design an algorithm which creates a linked list of all the nodes at each depth (e.g., if you have a tree with depth D, you'll have D linked lists)

# breadth first search from root, add nodes at each level to linked lists

def breadthFirstSearch_binary(tree):
    nodesAtLevel = [tree.root]
    while len(nodesAtLevel) > 0:
        nodesAtNextLevel = []
        yield nodesAtLevel
        for node in nodesAtLevel:
            if node:
                leftChild = node.getLeftChild()
                rightChild = node.getRightChild()
                if leftChild:
                    nodesAtNextLevel.append(leftChild)
                if rightChild:
                    nodesAtNextLevel.append(rightChild)
            
        nodesAtLevel = nodesAtNextLevel
        
def listOfDepths(tree):
    listOfLists = []
    for nodes in breadthFirstSearch_binary(tree):
        listOfLists.append(LinkedList([n.getData() for n in nodes]))
    return listOfLists

In [107]:
import unittest
from math import log
from collections import Counter
class TestListOfDepths(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.expectedArray = [[7],[4,14],[3,6,8]]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
            
    def test_listOfDepths(self):
        lists = listOfDepths(self.tree)
        lists = [l.getList() for l in lists]
        for l, exp in zip(lists, self.expectedArray):
            self.assertEqual(Counter(l), Counter(exp))
        
testSuiteListOfDepths = unittest.TestLoader().loadTestsFromTestCase(TestListOfDepths)
runner = unittest.TextTestRunner()
runner.run(testSuiteListOfDepths)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [119]:
# 4.4 Check Balanced: Implement a function to check if a binary tree is balanced. For the purposes of the question, a balanced tree is defined to be a tree such that the heights of the two subtrees of any node never differ by more than one.

# recursively compare heights of left and right subtrees 
# if difference is bigger than 1, false

def checkBalanced(tree):
    height, balanced = checkBalanced_recursive(tree.root)
    return balanced
# returns (height, balanced)
def checkBalanced_recursive(node):
    if not node:
        return (0, True)
    leftHeight, leftBalanced = checkBalanced_recursive(node.getLeftChild())
    rightHeight, rightBalanced = checkBalanced_recursive(node.getRightChild())
    balanced = leftBalanced and rightBalanced
    if abs(leftHeight - rightHeight) > 1:
        balanced = False
    return (max(leftHeight, rightHeight) + 1, balanced)

In [120]:
import unittest
class TestCheckBalanced(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.expectedArray = [[7],[4,14],[3,6,8]]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
            
    def test_checkBalanced(self):
        self.assertTrue(checkBalanced(self.tree))
        self.tree.add(1)
        self.tree.add(0)
        self.assertFalse(checkBalanced(self.tree))
        
testSuiteCheckBalanced = unittest.TestLoader().loadTestsFromTestCase(TestCheckBalanced)
runner = unittest.TextTestRunner()
runner.run(testSuiteCheckBalanced)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [129]:
# 4.5 Validate BST: Implement a function to check if a binary tree is a binary search tree.

# check if max of left subtree is less than or equal to current node's data and min of right subtree is bigger than current n
# recursively check for left and right
# return whether valid bst, min and max of tree

def validateBST(tree):
    valid, minimum, maximum = validateBST_recursive(tree.root)
    return valid
def validateBST_recursive(node):
    if not node:
        return (True, None, None)
    leftValidBST, leftMin, leftMax = validateBST_recursive(node.getLeftChild())
    rightValidBST, rightMin, rightMax = validateBST_recursive(node.getRightChild())
    valid = True
    
    minimum = leftMin
    if leftMin is None:
        minimum = node.getData()
    maximum = rightMax
    if rightMax is None:
        maximum = node.getData()
    if leftMax is not None:
        if leftMax > node.getData():
            valid = False
    if rightMin is not None:
        if rightMin < node.getData():
            valid = False
            
    return (valid, minimum, maximum)

In [130]:
import unittest
class TestValidateBST(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
            
    def test_validateBST(self):
        self.assertTrue(validateBST(self.tree))
        node = self.tree.root
        while node.getRightChild():
            node = node.getRightChild()
        newNode = BinaryTreeNode(0)
        node.setLeftChild(newNode)
        self.assertFalse(validateBST(self.tree))
        
testSuiteValidateBST = unittest.TestLoader().loadTestsFromTestCase(TestValidateBST)
runner = unittest.TextTestRunner()
runner.run(testSuiteValidateBST)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [150]:
# 4.6 Successor: Write an algorithm to find the "next" node (i.e., in-order successor) of a given node in a binary search tree. You may assume that each node has a link to its parent.

# next one should be left most node in right subtree
# if right child is null, can also look at left most node of parent's parent's right subtree
# the second case only works if the node's parent is the left child of the node's grandparent

def successor(node):
    rightChild = node.getRightChild()
    successorNode = None
    if rightChild:
        node = rightChild
        while node.getLeftChild():
            node = node.getLeftChild()
        successorNode = node
    else:
        parent = node.getParent()
        if parent:
            if parent.getLeftChild() == node:
                successorNode = parent
            else:
                grandparent = parent.getParent()
                if grandparent and grandparent.getLeftChild() == parent:
                    node = grandparent.getRightChild()
                    while node.getLeftChild():
                        node = node.getLeftChild()
                    successorNode = node
    return successorNode

In [151]:
import unittest
class TestSuccessor(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
            
    def test_successor(self):
        node = self.tree.root
        self.assertEqual(successor(node).getData(), 8)
        node = node.getLeftChild()
        self.assertEqual(successor(node).getData(), 6)
        node = node.getLeftChild()
        self.assertEqual(successor(node).getData(), 4)
        node = node.getParent().getRightChild()
        self.assertEqual(successor(node).getData(), 8)
        
testSuiteSuccessor = unittest.TestLoader().loadTestsFromTestCase(TestSuccessor)
runner = unittest.TextTestRunner()
runner.run(testSuiteSuccessor)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [180]:
# 4.7 Build Order: You are given a list of projects and a list of dependencies (which is a list of pairs of projects, where the first project is dependent on the second project). All of a project's dependecies must be built before the project is. Find a build order that will allow the projects to be built. If there is no valid build order, return an error.

# Example 
# Input: 
# projects: a,b,c,d,e,f
# dependencies: (d,a), (b,f), (d,b), (a,f), (c,d)
# Output: f,e,a,b,d,c

# topological sort
# source: https://en.wikipedia.org/wiki/Topological_sorting
# Kahn's algorithm
# set S of nodes with no incoming edges
# array L empty to contain nodes in topological order
# while S is not empty
#    remove node n from S and add it to end of L
#    for each edge from node n to node m
#        remove edge from graph
#        if node m has no incoming edges, add to S
# if graph still has edges
#    graph has at least one cycle
# else
#    return L (topologically sorted)

def topologicalSort(graph):
    sortedNodes = []
    
    edges = graph.edges()
    sourceNodes = graph.nodes()
    for _, target in edges:
        try:
            sourceNodes.remove(target)
        except ValueError:
            pass
    while len(sourceNodes) > 0:
        node = sourceNodes.pop()
        sortedNodes.append(node)
        edgesFromNode = graph.edges(node)
        for edge in edgesFromNode:
            _, target = edge
            graph.removeEdge(edge)
            edges = graph.edges()
            targetHasIncomingEdges = False
            for _, tempTarget in edges:
                if tempTarget == target:
                    targetHasIncomingEdges = True
                    break
            if not targetHasIncomingEdges:
                sourceNodes.append(target)
    edges = graph.edges()
    if len(edges) > 0:
        raise ValueError('graph has at least one cycle')
    else:
        return sortedNodes

def buildOrder(projects, dependencies):
    graph = Graph(dependencies, directed=True)
    sortedNodes = topologicalSort(graph)
    for project in projects:
        if project not in sortedNodes:
            sortedNodes.append(project)
            #independent projects can be started immediately
    return list(reversed(sortedNodes))

In [181]:
import unittest
class TestBuildOrder(unittest.TestCase):
    def setUp(self):
        self.dependencies = [('d','a'), ('b','f'), ('d','b'), ('a','f'), ('c','d')]
        self.projects = ['a','b','c','d','e','f']    
    def test_buildOrder(self):
        sortedProjects = buildOrder(self.projects, self.dependencies)
        for dependency in dependencies:
            dependent, independent = dependency
            indexOfDependent = sortedProjects.index(dependent)
            indexOfIndependent = sortedProjects.index(independent)
            self.assertTrue(indexOfIndependent < indexOfDependent)
        for project in self.projects:
            self.assertTrue(project in sortedProjects)
        
testSuiteBuildOrder = unittest.TestLoader().loadTestsFromTestCase(TestBuildOrder)
runner = unittest.TextTestRunner()
runner.run(testSuiteBuildOrder)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [211]:
# 4.8 First Common Ancestor: Design an algorithm and write code to find the first common ancestor of two nodes in a binary tree. Avoid storing additional nodes in a data structure. NOTE: This is not necessarily a binary search tree.

# if nodes have pointers to parents, this problem would be the same as a previous problem to find the intersection of two linked lists
# let's do this without pointers to parents
# search binary tree for node1 and node2
# if node1 and node2 are in different subtrees at a single node, that node is the common ancestor
# return node1 if node1 is contained in subtree
# return node2 if node2 is contained in subtree
# return commonAncestor if node1 and node 2 are in different subtrees or if subtree has commonAncestor
# return None if subtree doesn't have node1, node2, or the common ancestor

def firstCommonAncestor(tree, node1, node2):
    node, ancestorFound = firstCommonAncestor_recursive(tree.root, node1, node2)
    if ancestorFound:
        return node
    else:
        return None

def firstCommonAncestor_recursive(currentNode, node1, node2):
    if currentNode is None:
        return (None, False)
    data = currentNode.getData()
    leftResult, ancestorFoundLeft = firstCommonAncestor_recursive(currentNode.getLeftChild(), node1, node2)
    rightResult, ancestorFoundRight = firstCommonAncestor_recursive(currentNode.getRightChild(), node1, node2)
    
    if ancestorFoundRight:
        #ancestor found from right
        return (rightResult, True)
    if ancestorFoundLeft:
        return (leftResult, True)
    
    node1Found = data == node1 or leftResult == node1 or rightResult == node2
    node2Found = data == node2 or leftResult == node2 or rightResult == node2
    
    if node1Found and node2Found:
        return (data, True)
    if node1Found:
        return (node1, False)
    if node2Found:
        return (node2, False)
    return (None, False)

In [214]:
import unittest
class TestFirstCommonAncestor(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
            
    def test_firstCommonAncestor(self):
        self.assertEqual(firstCommonAncestor(self.tree, 3, 6), 4)
        self.assertEqual(firstCommonAncestor(self.tree, 3, 4), 4)
        self.assertEqual(firstCommonAncestor(self.tree, 6, 8), 7)
        self.assertTrue(firstCommonAncestor(self.tree, 16, 7) is None)
        
testSuiteFirstCommonAncestor = unittest.TestLoader().loadTestsFromTestCase(TestFirstCommonAncestor)
runner = unittest.TextTestRunner()
runner.run(testSuiteFirstCommonAncestor)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [210]:
# source: https://stackoverflow.com/questions/1732438/how-do-i-run-all-python-unit-tests-in-a-directory
testmodules = [
    TestGraph,
    TestSearch,
    TestRouteBetweenNodes, 
    TestBinaryTreeNode,
    TestBinaryTree,
    TestMinimalTree,
    TestListOfDepths,
    TestCheckBalanced,
    TestValidateBST,
    TestSuccessor,
    TestBuildOrder,
    TestFirstCommonAncestor
    ]

suite = unittest.TestSuite()

for t in testmodules:
    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(t))

unittest.TextTestRunner().run(suite)

..........................
----------------------------------------------------------------------
Ran 26 tests in 0.052s

OK


<unittest.runner.TextTestResult run=26 errors=0 failures=0>